# In-prompt chat completion

<a target="_blank" href="https://colab.research.google.com/github/masci/banks/blob/main/cookbook/in_prompt_completion.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Being able to build a prompt dynamically is instrumental for many use cases,
from retrieval to few-shots prompting, and very often an LLM can help you
generating the text used to render a template.

Banks provides the tools to use an LLM directly during the template rendering,
reducing the roundtrips needed to create the final prompt.

In [ ]:
!pip install banks

Let's gather some document that will be used later in the prompt, in this case a plain text version of the "Prompt Engineering" page on Wikipedia

In [ ]:
import requests

url = 'https://gist.githubusercontent.com/masci/3de92dbe61ba1e78b3f7ccde964dd339/raw/b8339f463cd3dc5397cb23080d89318dca7753ec/prompt_engineering.txt'
r = requests.get(url)
document = r.text

Let's implement prompt chaining with the help of an LLM:
- The first part of the prompt will create a list of quotes calling GPT-4o
- We use the list of quotes from the previous step to build the second part of the prompt
- We use GPT-4o to run the second prompt and get the final answer

To do this we'll use the `{% completion %}` tag and will assign its output to local template variables,
let's see how.

In [ ]:
import os
from banks import Prompt

## Banks will call OpenAI already when creating the Prompt object, so we need to set this upfront
os.environ["OPENAI_API_KEY"] = ""

p = Prompt("""
{# First prompt generate the quotes #}
{% set quotes %}
{% completion model="gpt-4o" %}
  {% chat role="system" %}
    You are a helpful assistant. Your task is to help answer a question given in a document.
    The first step is to extract quotes relevant to the question from the document,
    delimited by ####. Please output the list of quotes using <quotes></quotes>.
    Respond with "No relevant quotes found!" if no relevant quotes were found.
  {% endchat %}
  {% chat role="user" %}
    ####
    {{ document }}
    ####
  {% endchat %}
{% endcompletion %}
{% endset %}

{# Second and final prompt will use the quotes and do the actual query #}
{% set final %}
{% completion model="gpt-4o" %}
{% chat role="system" %}
Given a set of relevant quotes (delimited by <quotes></quotes>) extracted from
a document and the original document (delimited by ####), please compose an
answer to the question. Ensure that the answer is accurate, has a friendly tone,
and sounds helpful.
{% endchat %}

{% chat role="user" %}
####
{{ document }}
####
{{ quotes }}

{{ question }}
{% endchat %}
{% endcompletion %}
{% endset %}

{# We output the final result #}
{{ final }}
""")

print(p.text({"question": "What is In-context learning? Limit the answer to one sentence.", "document": document}))